In [1]:
###
# The goal of this notebook is to:
# - Take two neural populations
# - Compute the JS divergence between stimuli pairs for each population (the same stimuli pairs)
# - Compute the mutual information between the distributions of JS divergences

import glob
import os
from importlib import reload
import pickle
import datetime

import numpy as np
import scipy as sp
import pandas as pd
import h5py as h5
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import neuraltda.topology2 as tp2
import neuraltda.spectralAnalysis as sa
import neuraltda.simpComp as sc
import pycuslsa as pyslsa

daystr = datetime.datetime.now().strftime('%Y%m%d')
figsavepth = '/home/brad/DailyLog/'+daystr+'/'
print(figsavepth)

/home/brad/.conda/envs/neuraltda-4/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


/home/brad/DailyLog/20190314/


In [8]:
# Set up birds and block_paths
birds = ['B1083', 'B1056', 'B1235', 'B1075']
bps = {'B1083': '/home/brad/krista/B1083/P03S03/', 'B1075': '/home/brad/krista/B1075/P01S03/',
       'B1235': '/home/brad/krista/B1235/P02S01/', 'B1056': '/home/brad/krista/B1056/klusta/phy020516/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/',
       'B1056': '/home/brad/krista/B1056/klusta/phy020516/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/',
       'B1083-5': '/home/brad/krista/B1083/P03S05/'}


learned_stimuli = {'B1083': ['M_scaled_burung', 'N_scaled_burung', 'O_scaled_burung', 'P_scaled_burung'], 'B1056': ['A_scaled_burung', 'B_scaled_burung', 'C_scaled_burung', 'D_scaled_burung'], 'B1235': [], 'B1075': []}
peck_stimuli = {'B1083': {'L': ['N_40k','P_40k'], 'R': ['M_40k', 'O_40k']}, 'B1056': {'L': ['B_scaled_burung', 'D_scaled_burung'], 'R': ['A_scaled_burung', 'C_scaled_burung']}, 
                'B1235': {'L': ['F_scaled_burung', 'H_scaled_burung'], 'R': ['E_scaled_burung', 'G_scaled_burung'],}, 'B1075': {'L': ['F_40k', 'H_40k'], 'R': ['E_40k', 'G_40k']},
               'B1083-5': {'L': ['N_40k','P_40k'], 'R': ['M_40k', 'O_40k']}}

unfamiliar_stimuli = {'B1083': ['I_40k', 'J_40k', 'K_40k', 'L_40k'], 
                      'B1083-5': ['I_40k', 'J_40k', 'K_40k', 'L_40k'],
                      'B1235': ['A_scaled_burung', 'B_scaled_burung', 'C_scaled_burung', 'D_scaled_burung'], 
                      'B1075': ['A_40k', 'B_40k', 'C_40k', 'D_40k'], 
                      'B1056': ['E_scaled_burung', 'F_scaled_burung', 'G_scaled_burung', 'H_scaled_burung']
                     }

#bps =  {'B1056': '/home/AD/btheilma/krista/B1056/klusta/phy020516/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/',
#        'B1235': '/home/AD/btheilma/krista/B1235/P02S01/'}
#test_birds = ['B1056', 'B1235']
#test_birds = ['B1075', 'B1235']
#test_birds = ['B1056', 'B1235']
#test_birds =['B1056', 'B1083']
#test_birds = ['B1083']
#test_birds = ['B1083', 'B1083-5']
test_birds = ['B1056', 'B1235', 'B1083', 'B1083-5']
test_birds = ['B1056']
# Binning Parameters
windt = 10.0                      # milliseconds
dtovr = 0.5*windt                 # milliseconds
segment_info = [0, 0]             # use full Trial
cluster_group = ['Good']          # use just good clusters
comment = 'JS_MI_TEST'            # BootStrap Populations
bdfs = {}                         # Dictionary to store bdf

In [9]:
# Loop through each bird in our list and bin the data
for bird in test_birds:
    block_path = bps[bird]
    bfdict = tp2.dag_bin(block_path, windt, segment_info, cluster_group=cluster_group, dt_overlap=dtovr, comment=comment)
    bdf = glob.glob(os.path.join(bfdict['raw'], '*.binned'))[0]
    print(bdf)
    bdfs[bird] = bdf

/home/brad/krista/B1056/klusta/phy020516/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/binned_data/win-10.0_dtovr-5.0_seg-0-0-JS_MI_TEST/20180525T180834Z-10.0-5.0.binned


In [10]:
# extract left vs right stims
# extract population tensors for the populations of interest
# Do not sort the stims
population_tensors_familiar = {}
stimuli = []

for bird in test_birds:
    stimuli = peck_stimuli[bird]['L'] + peck_stimuli[bird]['R']
    print(stimuli)
    bdf = bdfs[bird]
    population_tensors_familiar[bird] = []
    # open the binned data file
    with h5.File(bdf, 'r') as f:
        #stimuli = f.keys()
        print(list(f.keys()))
        for stim in stimuli:
            poptens = np.array(f[stim]['pop_tens'])
            population_tensors_familiar[bird].append([poptens, stim])

['B_scaled_burung', 'D_scaled_burung', 'A_scaled_burung', 'C_scaled_burung']
['A_scaled_burung', 'B_scaled_burung', 'C_scaled_burung', 'D_scaled_burung', 'E_scaled_burung', 'F_scaled_burung', 'G_scaled_burung', 'H_scaled_burung', 'I_scaled_burung', 'J_scaled_burung', 'K_scaled_burung', 'L_scaled_burung', 'M_scaled_burung', 'N_scaled_burung', 'O_scaled_burung', 'P_scaled_burung']


In [11]:
# extract Unfamiliar stims
# extract population tensors for the populations of interest
# Do not sort the stims
population_tensors_unfamiliar = {}
stimuli = []

for bird in test_birds:
    stimuli = unfamiliar_stimuli[bird]
    print(stimuli)
    bdf = bdfs[bird]
    population_tensors_unfamiliar[bird] = []
    # open the binned data file
    with h5.File(bdf, 'r') as f:
        #stimuli = f.keys()
        print(list(f.keys()))
        for stim in stimuli:
            poptens = np.array(f[stim]['pop_tens'])
            population_tensors_unfamiliar[bird].append([poptens, stim])

['E_scaled_burung', 'F_scaled_burung', 'G_scaled_burung', 'H_scaled_burung']
['A_scaled_burung', 'B_scaled_burung', 'C_scaled_burung', 'D_scaled_burung', 'E_scaled_burung', 'F_scaled_burung', 'G_scaled_burung', 'H_scaled_burung', 'I_scaled_burung', 'J_scaled_burung', 'K_scaled_burung', 'L_scaled_burung', 'M_scaled_burung', 'N_scaled_burung', 'O_scaled_burung', 'P_scaled_burung']


In [17]:
# flatten the list of population tensors for each population
threshold = 6

def threshold_poptens(tens, thresh):
    ncell, nwins, ntrials = tens.shape
    frs = np.mean(tens, axis=1)
    tfr = thresh*frs
    tfrtens = np.tile(tfr[:, np.newaxis, :], (1, nwins, 1))
    bintens = 1*np.greater(tens, tfrtens)
    return bintens

def shuffle_binmat(binmat):
    ncells, nwin = binmat.shape
    for i in range(ncells):
        binmat[i, :] = np.random.permutation(binmat[i, :])
    return binmat

def get_JS(i, j, Li, Lj, speci, specj, beta):
    js = (i, j, sc.sparse_JS_divergence2_fast(Li, Lj, speci, specj, beta))
    print((i, j))
    return js

In [13]:

poptens = {'familiar': population_tensors_familiar, 'unfamiliar': population_tensors_unfamiliar}

In [ ]:
# Compute JS popA:
#Left vs right

dim = 1
beta = 1.0

for bird in test_birds:
    for sh in ['original', 'shuffled']:
        for fam in ['familiar', 'unfamiliar']:
            ntrials = 10 # Only do half the trials for each stim
            bird_tensors = poptens[fam][bird]
            SCG = []
            spectra = []
            for bird_tensor, stim in bird_tensors:
                binmatlist = []
                print(bird, stim)
                ncells, nwin, _ = bird_tensor.shape
                bin_tensor = threshold_poptens(bird_tensor, threshold)
                for trial in tqdm(range(ntrials)):
                    if sh == 'shuffled':
                        binmatlist.append(shuffle_binmat(bin_tensor[:, :, trial]))
                    else:
                        binmatlist.append(bin_tensor[:, :, trial])
                    ms = sc.binarytomaxsimplex(bin_tensor[:, :, trial], rDup=True)
                    scg1 = sc.simplicialChainGroups(ms)
                    SCG.append(scg1)
                    L = sc.sparse_laplacian(scg1, dim)
                    rho = sc.sparse_density_matrix(L, beta)
                    r = sc.sparse_density_spectrum(rho)
                    spectra.append(r)
            N = len(SCG)
            # compute density matrices
            
            jsmat = np.zeros((N, N))
            for i in tqdm(range(N)):
                for j in range(i, N):
                    jsmat[i, j] = sc.sparse_reconciled_spectrum_JS(spectra[i], spectra[j])
            with open(os.path.join(figsavepth, 'JSpop_{}-{}-{}-{}_LvsR-{}-{}'.format(bird, dim, beta, ntrials, fam, sh)), 'wb') as f:
                pickle.dump(jsmat, f)
            


In [25]:
# mirroring cuda code
#Left vs right
reload(sc)
dim = 1
beta = 3

betas = [1]

for bird in test_birds:
    for sh in ['original']:
        for fam in ['familiar']:
            ntrials = 5 # Only do half the trials for each stim
            bird_tensors = poptens[fam][bird]
            SCG = []
            spectra = []
            laplacians = []
            print('Computing Laplacians and Spectra for {} {} {}...'.format(bird, sh, fam))
            for bird_tensor, stim in bird_tensors:
                binmatlist = []
                print(bird, stim)
                ncells, nwin, _ = bird_tensor.shape
                bin_tensor = threshold_poptens(bird_tensor, threshold)
                for trial in tqdm(range(ntrials)):
                    if sh == 'shuffled':
                        binmatlist.append(shuffle_binmat(bin_tensor[:, :, trial]))
                    else:
                        binmatlist.append(bin_tensor[:, :, trial])
                    ms = sc.binarytomaxsimplex(bin_tensor[:, :, trial], rDup=True)
                    scg1 = sc.simplicialChainGroups(ms)
                    SCG.append(scg1)
                    L = sc.sparse_laplacian(scg1, dim)
#                     rho = sc.sparse_density_matrix(L, beta)
#                     r = sc.sparse_density_spectrum(rho)
                    laplacians.append(L)
                    spectra.append(sc.sparse_spectrum(L))
            N = len(SCG)
            # compute density matrices
            pairs = [(i, j) for i in range(N) for j in range(i, N)]
            for beta in betas:
                print('Computing JS Divergences with beta {}...'.format(beta))
                jsmat = np.zeros((N, N))
                
                jsdat = Parallel(n_jobs=23)(delayed(get_JS)(i, j, laplacians[i], laplacians[j], spectra[i], spectra[j], beta) for (i, j) in pairs)
                for d in jsdat:
                    jsmat[d[0], d[1]] = d[2]
            
                with open(os.path.join(figsavepth, 'JSpop_fast_{}-{}-{}-{}_LvsR-{}-{}.pkl'.format(bird, dim, beta, ntrials, fam, sh)), 'wb') as f:
                    pickle.dump(jsmat, f)
            


Computing Laplacians and Spectra for B1056 original familiar...
B1056 B_scaled_burung


B1056 D_scaled_burung


B1056 A_scaled_burung


B1056 C_scaled_burung


Exception in thread Thread-41:
Traceback (most recent call last):
  File "/home/brad/.conda/envs/neuraltda-4/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/brad/.conda/envs/neuraltda-4/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/brad/.conda/envs/neuraltda-4/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration




Computing JS Divergences with beta 1...
(0, 0)
(0, 4)
(0, 3)
(0, 2)
(1, 1)
(0, 1)
(0, 10)
(1, 3)
(1, 2)
(1, 4)
(0, 7)
(0, 6)
(0, 11)
(0, 8)
(0, 14)
(0, 16)
(0, 12)
(1, 10)
(0, 17)
(0, 18)
(1, 7)
(1, 11)
(1, 6)
(0, 13)
(0, 19)
(0, 15)
(1, 8)
(2, 2)
(0, 9)
(2, 8)
(1, 12)
(2, 4)
(0, 5)
(2, 3)
(1, 14)
(1, 16)
(1, 15)
(1, 5)
(1, 9)
(2, 10)
(3, 3)
(1, 17)
(1, 13)
(2, 6)
(1, 18)
(2, 11)
(2, 7)
(3, 4)
(2, 15)
(2, 14)
(2, 16)
(2, 12)
(1, 19)
(2, 9)
(3, 10)
(2, 18)
(3, 8)
(3, 7)
(2, 13)
(4, 4)
(2, 17)
(3, 6)
(3, 11)
(2, 5)
(2, 19)
(3, 14)
(3, 16)
(3, 5)
(3, 12)
(4, 6)
(4, 10)
(4, 7)
(3, 15)
(4, 8)
(3, 17)
(3, 9)
(3, 13)
(3, 18)
(4, 11)
(4, 16)
(3, 19)
(4, 9)
(4, 13)
(4, 14)
(4, 12)
(4, 18)
(4, 5)
(6, 10)
(4, 17)
(4, 19)
(4, 15)
(6, 6)
(6, 11)
(6, 8)
(5, 18)
(5, 5)
(6, 7)
(5, 10)
(5, 11)
(5, 19)
(5, 6)
(5, 8)
(5, 15)
(5, 7)
(5, 12)
(5, 16)
(5, 9)
(5, 14)
(6, 16)
(5, 17)
(6, 9)
(5, 13)
(6, 14)
(7, 7)
(6, 15)
(6, 12)
(7, 8)
(6, 13)
(7, 10)
(7, 11)
(7, 14)
(6, 18)
(6, 17)
(7, 16)
(8, 8)
(7, 12)
(6,

In [15]:
from joblib import Parallel, delayed


In [ ]:
test